<a href="https://colab.research.google.com/github/FBehrad/Charm/blob/main/Charm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Get requirements.py


In [ ]:
!wget https://raw.githubusercontent.com/FBehrad/Charm/refs/heads/main/requirements.txt



---


2. Install the rquirements (***Extremly important***):

**Note:** If the following cell didn't work, you should install packages one by one:

For example:

```
!pip install timm==0.4.12
```



In [ ]:
!pip install -r requirements.txt




---
3. Install Charm tokenizer


In [ ]:
!pip install Charm-tokenizer==1.0.16



---
4. Prepare patches and their corresponding position embeddings


In [1]:
from Charm_tokenizer.ImageProcessor import Charm_Tokenizer
import requests

url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
img_path = requests.get(url, stream=True).raw

charm_tokenizer = Charm_Tokenizer(patch_selection='frequency', training_dataset='tad66k', without_pad_or_dropping=True)
tokens, pos_embed, mask_token = charm_tokenizer.preprocess(img_path)



---
5. Predict aesthetic score


In [2]:
from Charm_tokenizer.Backbone import backbone
from Charm_tokenizer import model

model = backbone(training_dataset='tad66k', device='cpu')
prediction = model.predict(tokens, pos_embed, mask_token)

Prediction: 3.813 from 1 to 10
